In [16]:
from selenium import webdriver
from lxml import etree
import re, time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [27]:
class LagouSpider(object):
    driver_path = '/Users/myki/Documents/Python/chromedriver'
    
    def __init__(self):
        self.driver = webdriver.Chrome(LagouSpider.driver_path)
        self.url = 'https://www.lagou.com/jobs/list_python?labelWords=&fromSearch=true&suginput='
        self.positions = []
    
    def run(self):
        self.driver.get(self.url)
        while True:
            source = self.driver.page_source
            self.parse_list_page(source)
            WebDriverWait(self.driver, timeout=10).until(
                EC.presence_of_element_located((By.XPATH, "//div[@class='pager_container']/span[last()]"))
            )
            next_btn = self.driver.find_element_by_xpath("//div[@class='pager_container']/span[last()]")
            if "pager_next pager_next_disabled" in next_btn.get_attribute('class'):
                break
            else:
                next_btn.click()
            time.sleep(1)
        
    def parse_list_page(self, source):
        html = etree.HTML(source)
        links = html.xpath("//a[@class = 'position_link']/@href")
        for link in links:
            self.request_detail_page(link)
            time.sleep(1)
            
    def request_detail_page(self, url):
        #在另一个页面打开
        self.driver.execute_script("window.open('%s')" % url)
        #切换至新打开的页面
        self.driver.switch_to.window(self.driver.window_handles[1])
        
        WebDriverWait(self.driver, timeout=10).until(
           EC.presence_of_element_located((By.XPATH, "//span[@class='name']"))
        )
        source = self.driver.page_source
        self.parse_detail_page(source)
        
        #关闭当前详情页面
        self.driver.close()
        #切换回职位列表页面
        self.driver.switch_to.window(self.driver.window_handles[0])
        
    def parse_detail_page(self, source):
        html = etree.HTML(source)
        position_name = html.xpath("//span[@class = 'name']/text()")[0]
        job_request_spans = html.xpath("//dd[@class = 'job_request']//span")
        salary = job_request_spans[0].xpath(".//text()")[0].strip()
        city = job_request_spans[1].xpath(".//text()")[0].strip()
        city = re.sub(r"[\s/]", "", city)
        work_years = job_request_spans[2].xpath(".//text()")[0].strip()
        work_years = re.sub(r"[\s/]", "", work_years)
        education = job_request_spans[3].xpath(".//text()")[0].strip()
        education = re.sub(r"[\s/]", "", education)
        desc = "".join(html.xpath("//dd[@class = 'job_bt']//text()")).strip()
        company_name = html.xpath("//h3[@class='fl']/em/text()")[0].strip()
        
        position = {
            'Name': position_name,
            'Company_name': company_name,
            'Salary': salary,
            'City': city,
            'Work_years': work_years,
            'Education': education,
            'Desc': desc
        }
        self.positions.append(position)
        print(position)
        print('*'*30 + '\n')
    
if __name__ == '__main__':
    spider = LagouSpider()
    spider.run()

{'Name': '糗百招聘', 'Company_name': '糗事百科', 'Salary': '15k-30k', 'City': '深圳', 'Work_years': '经验3-5年', 'Education': '本科及以上', 'Desc': '职位描述：\n        \n        工作内容：\n参与糗事百科的后端研发；\n保障产品的快速迭代与服务质量；\n任职资格：\n三年以上高并发系统的设计与调优经验，对系统的架构与实现有清晰的认识；\n熟悉linux系统生产环境搭建与调优；\n至少熟悉 Python/Golang/C/C++ 其中一种语言\n熟练掌握Mysql数据库的使用与优化；\n熟悉至少一种web框架的使用和调优，并了解其基本原理\n扎实的编程功底，能享受编程乐趣；\n高效的学习能力和分析解决问题能力；\n本科及以上学历；\n\n加分项：有团队管理经验，团队驱动力者优先~ ~'}
******************************

{'Name': '圆通科技HR', 'Company_name': '圆通', 'Salary': '15k-25k', 'City': '上海', 'Work_years': '经验3-5年', 'Education': '本科及以上', 'Desc': '职位描述：\n        \n        岗位职责：\n1.参与自动化运维平台开发和建设\n2.参与自动化发布平台CI\\CD开发和建设\n3.参与统一监控平台开发和建设\n4.参与运维工具开发和建设\n5.参与cmdb开发和建设\n\n任职要求：\n1.精通Python开发，3年以上Python开发经验，熟悉基于Python代码的性能分析和优化\n2.精通Flask/Django或者其他Python Web开发框架，熟悉MVC架构，熟悉常用设计模式\n3.熟练使用mysql，熟悉mysql的各种存储引擎，熟悉索引工作原理；\n4.熟悉 celery 等队列工具的开发使用；\n5.有运维自动化、监控系统、应用发布系统、CMDB配置管理系统等运维开发或者维护经验优先；\n6.熟悉开源监控软件Nagios/Zabbix/Cacti/Prometheus一种或以上的部署和应用经验优先\n7.熟悉 Web 前端技能，有vue使用经验

{'Name': '李迎霜', 'Company_name': '灏程信息', 'Salary': '10k-20k', 'City': '上海', 'Work_years': '经验3-5年', 'Education': '大专及以上', 'Desc': '职位描述：\n        \n        岗位职责：\n1、负责公司用户端业务接口的功能开发和代码维护，能独立完成子系统和功能模块开发，编写完整的接口文档；\n2、根据需求制定技术方案，项目计划以及开发,学习研究最新技术，并将之应用到工作当中；\n3、根据产品需求，进行系统设计和编码, 持续对系统架构进行改造和优化；\n4、具备良好的编码习惯，结构清晰，命名规范，逻辑性强，代码冗余率低。\n岗位要求：\n1、3年及以上Python开发经验，熟练Flask，Django, Sanic, Sqlalchemy，Celery等主流框架, 深入理解框架实现原理及特性, 熟练使用python的常用库, 熟悉网络爬虫；\n2、熟悉Internet常用协议，如HTTP、TCP/IP、熟悉MySQL、Redis等，具备数据库应用、设计能力；\n3、理解并能应用RESTFul规范,具备较强的编程能力和良好的编码风格；\n4、熟练掌握Python语言及web相关技术栈；\n5、善于团队合作与管理，对新技术有强烈的探索欲望，责任心强，能够承受工作压力，有强烈的责任心；\n6、了解异步框架Async,集群与负载均衡,消息中间件等技术；\n7、不了解异步框架的请慎投，谢谢！'}
******************************



ElementClickInterceptedException: Message: element click intercepted: Element <span hidefocus="hidefocus" action="next" class="pager_next ">...</span> is not clickable at point (622, 785). Other element would receive the click: <div class="body-container showData">...</div>
  (Session info: chrome=79.0.3945.88)
